# PREPROCESAMIENTO - TEST

El objetivo del presente notebook consiste en la eliminación de variables previamente seleccionadas mediante análisis del diccionario de datos y tipo de datos por variable, junto con la dumización de las variables categóricas.

## SET UP

In [1]:
!pip install findspark

import findspark
findspark.init()

## LIBRERIAS

In [2]:
# Cargar Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("preprocessing").master("local[*]").getOrCreate()

In [3]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## CARGAR DATOS

In [5]:
df = spark.read.option("header","true").option("inferschema","true").option("delimiter", ";").option("nullValue", "\\N").csv("input/dt2020_base_evaluar_semicolon.csv")

In [6]:
df.count()

281666

## PREPROCESAMIENTO

### Variables seleccionadas para reducir columnas

In [7]:
lista = ['tiene_ctas_embargadas', 'rechazo_credito', 'pension_fopep', 'tiene_cred_hipo_2', 'tiene_cred_hipo_1', 'cuota_cred_hipot', 'tiene_consumo', 'tiene_crediagil', 'convenio_lib', 'ingreso_nomina', 'ingreso_segurida_social','ciudad_residencia','ciudad_laboral','departamento_laboral']

### Eliminación de dimensionalidad

In [8]:
df = df.drop(*lista)

### Variables seleccionadas para reducir registros

In [9]:
variables = ['ind','ingreso_final','cupo_total_tc','cuota_tc_bancolombia','mediana_nom3','cuota_de_consumo','cuota_tarjeta_de_credito','cupo_tc_mdo','saldo_prom3_tdc_mdo','cuota_tc_mdo','saldo_no_rot_mdo']

In [10]:
data_preprocessed = df

## One hot encoding

In [11]:
categorical_variables = ['genero','estado_civil','nivel_academico','profesion','ocupacion','tipo_vivienda','departamento_residencia','cat_ingreso','rep_calif_cred','ind_mora_vigente','cartera_castigada','tenencia_tc','tiene_ctas_activas']

In [12]:
data_preprocessed = data_preprocessed.fillna("NA")

In [13]:
dummy = "_dummy"
dumies = []
for var in categorical_variables:
    dumies.append(var + dummy)

In [14]:
indexer = "_indexer"
indexers = []
for var in categorical_variables:
    indexers.append(var + indexer)

In [15]:
indexed = data_preprocessed

for i in range(len(categorical_variables)):
    indexer = StringIndexer(inputCol = categorical_variables[i], outputCol=indexers[i])
    indexed = indexer.fit(indexed).transform(indexed)

In [16]:
df_encode = indexed

for i in range(len(categorical_variables)):
    encoder = OneHotEncoder(inputCol=indexers[i], outputCol=dumies[i])
    encoder = encoder.fit(df_encode)
    df_encode = encoder.transform(df_encode)

In [17]:
## clean variables
df_encode = df_encode.drop(*categorical_variables)
df_encode = df_encode.drop(*indexers)

In [18]:
df_encode.columns

['id_registro',
 'periodo',
 'id_cli',
 'fecha_nacimiento',
 'edad',
 'ult_actual',
 'categoria',
 'codigo_ciiu',
 'mora_max',
 'cant_moras_30_ult_12_meses',
 'cant_moras_60_ult_12_meses',
 'cant_moras_90_ult_12_meses',
 'cupo_total_tc',
 'cuota_tc_bancolombia',
 'nro_tot_cuentas',
 'ctas_activas',
 'ctas_embargadas',
 'mediana_nom3',
 'mediana_pen3',
 'ingreso_nompen',
 'ingreso_final',
 'cant_mora_30_tdc_ult_3m_sf',
 'cant_mora_30_consum_ult_3m_sf',
 'cuota_de_vivienda',
 'cuota_de_consumo',
 'cuota_rotativos',
 'cuota_tarjeta_de_credito',
 'cuota_de_sector_solidario',
 'cuota_sector_real_comercio',
 'cupo_tc_mdo',
 'saldo_prom3_tdc_mdo',
 'cuota_tc_mdo',
 'saldo_no_rot_mdo',
 'cuota_libranza_sf',
 'cant_oblig_tot_sf',
 'cant_cast_ult_12m_sr',
 'ind',
 'pol_centr_ext',
 'genero_dummy',
 'estado_civil_dummy',
 'nivel_academico_dummy',
 'profesion_dummy',
 'ocupacion_dummy',
 'tipo_vivienda_dummy',
 'departamento_residencia_dummy',
 'cat_ingreso_dummy',
 'rep_calif_cred_dummy',
 'ind_m

In [19]:
df_encode.count()

281666

In [21]:
df_encode.write.parquet("output/preprocessing/preprocessing_data_test.parquet")